## This notebook is for evaluating the performance of each server from the 3 servers ##
1. MySQL
2. MSSQL (Microsoft SQL Server)
3. PostgreSQL

### Importing the required libraries ###

In [1]:
import time
from dotenv import load_dotenv
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import random

### Setting the dataframe that will contain the results ###

In [2]:
servers_performance = pd.DataFrame(columns=["MySQL/INSERT", "MySQL/UPDATE", "MSSQL/INSERT", "MSSQL/UPDATE", "PostgreSQL/INSERT", "PostgreSQL/UPDATE"],
                                   index=range(1, 91))
servers_performance = servers_performance.astype(float)
servers_performance.index.name = "StatementID"

In [3]:
servers_performance

,MySQL/INSERT,MySQL/UPDATE,MSSQL/INSERT,MSSQL/UPDATE,PostgreSQL/INSERT,PostgreSQL/UPDATE
StatementID,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
86,NaN,NaN,NaN,NaN,NaN,NaN
87,NaN,NaN,NaN,NaN,NaN,NaN
88,NaN,NaN,NaN,NaN,NaN,NaN


### Prepare the data to be inserted ###

In [4]:
# Generate random data
prices = random.sample(range(1000, 10000), 90)
items_in_stock = random.sample(range(100, 1000), 90)
random_ids = random.sample(range(10, 100), 90)
product_ids = [f"P{num}" for num in random_ids]

products_data = pd.DataFrame({"ProductID": product_ids, "Price": prices, "ItemsInStock": items_in_stock},
                             index=range(1, 91))

products_data

,ProductID,Price,ItemsInStock
1,P72,9623,128
2,P80,9397,120
3,P79,2428,669
4,P49,7820,199
5,P57,1274,211
...,...,...,...
86,P31,2941,109
87,P92,5625,754
88,P95,4694,684
89,P19,2357,771


### MySQL Server ###

In [5]:
load_dotenv()
ENV_MYSQL_USER = os.getenv('MYSQL_USER')
ENV_MYSQL_PASSWORD = os.getenv('MYSQL_PASSWORD')
ENV_MYSQL_HOST = os.getenv('MYSQL_HOST')
ENV_MYSQL_DATABASE = os.getenv('MYSQL_DATABASE')

In [6]:
import mysql.connector as connector

# Connect to the database
connection = connector.connect(user=ENV_MYSQL_USER,
                            password=ENV_MYSQL_PASSWORD,
                            host=ENV_MYSQL_HOST,
                            database=ENV_MYSQL_DATABASE)

In [7]:
# Create a link "cursor" between MySQL and Python
cursor = connection.cursor()

In [8]:
cursor.execute("""DELETE FROM Products;""")

In [9]:
# For evaluating INSERT performance
for i in range(90):
    product_id, price, items_in_stock = products_data.iloc[i]
    add_record = """INSERT INTO Products VALUES('{}', {}, {});""".format(product_id, price, items_in_stock)

    start_time = time.time()
    cursor.execute(add_record)
    end_time = time.time()
    total_time = end_time - start_time
    total_time_in_msec = total_time*1000
    servers_performance.iloc[i, 0] = total_time_in_msec

    cursor.execute("""DELETE FROM Products;""")

    time.sleep(0.5)

# For evaluating UPDATE performance
for i in range(90): # Fill the database with data
    product_id, price, items_in_stock = products_data.iloc[i]
    add_record = """INSERT INTO Products VALUES('{}', {}, {});""".format(product_id, price, items_in_stock)
    cursor.execute(add_record)
    connection.commit()

for i in range(90): # Update randomly
    new_items_in_stock = 50
    product_id = products_data["ProductID"][i+1]
    update_record = """UPDATE Products SET ItemsInStock = {} WHERE ProductID = '{}';""".format(new_items_in_stock, product_id)

    start_time = time.time()
    cursor.execute(update_record)
    end_time = time.time()
    total_time = end_time - start_time
    total_time_in_msec = total_time*1000
    servers_performance.iloc[i, 1] = total_time_in_msec
    time.sleep(0.5)

In [10]:
servers_performance

,MySQL/INSERT,MySQL/UPDATE,MSSQL/INSERT,MSSQL/UPDATE,PostgreSQL/INSERT,PostgreSQL/UPDATE
StatementID,,,,,,
1,0.995874,0.997305,NaN,NaN,NaN,NaN
2,0.545025,1.059055,NaN,NaN,NaN,NaN
3,2.051830,1.077890,NaN,NaN,NaN,NaN
4,1.068592,1.089811,NaN,NaN,NaN,NaN
5,1.107454,1.104116,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
86,0.000000,28.957605,NaN,NaN,NaN,NaN
87,1.011848,1.025915,NaN,NaN,NaN,NaN
88,0.000000,2.008915,NaN,NaN,NaN,NaN


In [11]:
cursor.close()
connection.close()

### MSSQL ###

In [12]:
load_dotenv()
ENV_MSSQL_SERVER = os.getenv('MSSQL_SERVER')

ENV_MSSQL_DATABASE = os.getenv('MSSQL_DATABASE')

In [13]:
import pyodbc

# Connect to the database using Windows authentication
connection = pyodbc.connect('DRIVER={SQL Server};SERVER='+ENV_MSSQL_SERVER+';DATABASE='+ENV_MYSQL_DATABASE+';Trusted_Connection=yes;')

In [14]:
# Create a link "cursor" between MSSQL and Python
cursor = connection.cursor()

In [15]:
cursor.execute("""DELETE FROM Products;""")

In [16]:
# For evaluating INSERT performance
for i in range(90):
    product_id, price, items_in_stock = products_data.iloc[i]
    add_record = """INSERT INTO Products VALUES('{}', {}, {});""".format(product_id, price, items_in_stock)

    start_time = time.time()
    cursor.execute(add_record)
    end_time = time.time()
    total_time = end_time - start_time
    total_time_in_msec = total_time*1000
    servers_performance.iloc[i, 2] = total_time_in_msec

    cursor.execute("""DELETE FROM Products;""")

    time.sleep(0.5)

# For evaluating UPDATE performance
for i in range(90): # Fill the database with data
    product_id, price, items_in_stock = products_data.iloc[i]
    add_record = """INSERT INTO Products VALUES('{}', {}, {});""".format(product_id, price, items_in_stock)
    cursor.execute(add_record)
    connection.commit()

for i in range(90): # Update randomly
    new_items_in_stock = 50
    product_id = products_data["ProductID"][i+1]
    update_record = """UPDATE Products SET ItemsInStock = {} WHERE ProductID = '{}';""".format(new_items_in_stock, product_id)

    start_time = time.time()
    cursor.execute(update_record)
    end_time = time.time()
    total_time = end_time - start_time
    total_time_in_msec = total_time*1000
    servers_performance.iloc[i, 3] = total_time_in_msec
    time.sleep(0.5)


In [17]:
cursor.close()
connection.close()

### PostgreSQL ###

In [26]:
load_dotenv()
ENV_POSTGRES_USER = os.getenv('POSTGRES_USER')
ENV_POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
ENV_POSTGRES_HOST = os.getenv('POSTGRES_HOST')
ENV_POSTGRES_DATABASE = os.getenv('POSTGRES_DATABASE')
ENV_POSTGRES_PORT = os.getenv('POSTGRES_PORT')

In [27]:
import psycopg2

# Connect to the database
connection = psycopg2.connect(host=ENV_POSTGRES_HOST,
                                port=ENV_POSTGRES_PORT,
                                database=ENV_POSTGRES_DATABASE,
                                user=ENV_POSTGRES_USER,
                                password=ENV_POSTGRES_PASSWORD)

In [28]:
# Create a link "cursor" between PostgreSQL and Python
cursor = connection.cursor()

In [29]:
cursor.execute("""DELETE FROM Products;""")

In [30]:
# For evaluating INSERT performance
for i in range(90):
    product_id, price, items_in_stock = products_data.iloc[i]
    add_record = """INSERT INTO Products VALUES('{}', {}, {});""".format(product_id, price, items_in_stock)

    start_time = time.time()
    cursor.execute(add_record)
    end_time = time.time()
    total_time = end_time - start_time
    total_time_in_msec = total_time*1000
    servers_performance.iloc[i, 4] = total_time_in_msec

    cursor.execute("""DELETE FROM Products;""")

    time.sleep(0.5)

# For evaluating UPDATE performance
for i in range(90): # Fill the database with data
    product_id, price, items_in_stock = products_data.iloc[i]
    add_record = """INSERT INTO Products VALUES('{}', {}, {});""".format(product_id, price, items_in_stock)
    cursor.execute(add_record)
    connection.commit()

for i in range(90): # Update randomly
    new_items_in_stock = 50
    product_id = products_data["ProductID"][i+1]
    update_record = """UPDATE Products SET ItemsInStock = {} WHERE ProductID = '{}';""".format(new_items_in_stock, product_id)

    start_time = time.time()
    cursor.execute(update_record)
    end_time = time.time()
    total_time = end_time - start_time
    total_time_in_msec = total_time*1000
    servers_performance.iloc[i, 5] = total_time_in_msec
    time.sleep(0.5)


### Statistics ###

In [31]:
servers_performance

,MySQL/INSERT,MySQL/UPDATE,MSSQL/INSERT,MSSQL/UPDATE,PostgreSQL/INSERT,PostgreSQL/UPDATE
StatementID,,,,,,
1,0.995874,0.997305,4.993916,1.484394,0.000000,0.998735
2,0.545025,1.059055,0.000000,1.077414,0.000000,0.488043
3,2.051830,1.077890,2.022028,1.015186,1.019955,0.000000
4,1.068592,1.089811,0.000000,0.000000,1.057863,0.508070
5,1.107454,1.104116,1.066923,1.393557,0.000000,0.000000
...,...,...,...,...,...,...
86,0.000000,28.957605,2.016544,2.010345,0.519514,0.000000
87,1.011848,1.025915,2.040863,0.502348,1.007080,1.067877
88,0.000000,2.008915,1.287460,0.571966,0.570774,0.225782


In [32]:
servers_performance.describe()

,MySQL/INSERT,MySQL/UPDATE,MSSQL/INSERT,MSSQL/UPDATE,PostgreSQL/INSERT,PostgreSQL/UPDATE
count,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000
mean,1.096781,1.964042,1.455720,1.062783,0.600012,0.728459
std,0.753608,7.567914,2.009794,1.525527,0.484151,0.540188
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.516593,0.983894,0.737488,0.350356,0.000000,0.182867
50%,1.049161,1.060128,1.060367,0.867367,0.628233,1.017451
75%,2.008915,1.079798,1.847744,1.058936,1.022816,1.061141
max,2.387524,67.147970,17.133951,10.476112,1.282454,2.069473
